# Processing Site Lidar

In this notebook we preprocess out lidar sites. This includes:
- Reprojection
- Cropping to the site geometry
- Computing height above ground
- Remove statistical outliers (noise)
- Remove manually labelled noise (cloud noise)
- Loading height above ground into Z coordinate
- Clamping minimum Z to 0
- Saving cloud as a cloud optimised point cloud (COPC)

To process the data we will use PDAL pipelines.
We will also use dask to run the processing in parallel.

In [1]:
from pathlib import Path
import json

import geopandas as gpd
import pdal
import pandas as pd

## PDAL Pipeline Template

In [2]:
# Note, we could have this just a string, but as a dict allows us to add comments
def create_site_pipeline(
    input_path: str = '',
    output_path: str = '',
    polygon_wkt: str = '',
    manual_pre_norm_noise_expr: str | None = None,
    manual_post_norm_noise_expr: str | None = None,
) -> str:

    pre_hag_filter = (
        {"type": "filters.assign", "value": manual_pre_norm_noise_expr}
        if manual_pre_norm_noise_expr is not None
        else None
    )

    post_hag_filter = (
        {"type": "filters.assign", "value": manual_post_norm_noise_expr}
        if manual_post_norm_noise_expr is not None
        else None
    )

    pipeline_template_dict = [
        # Read the input LAS file
        {"type": "readers.las", "filename": input_path},
        # Reproject to MGA2020 + Aus Height Datum
        {"type": "filters.reprojection", "out_srs": "EPSG:7855+5711"},
        # Crop to our site polygon
        # This is optional, but useful as our source sites are quite large
        # and we don't need the whole thing
        {"type": "filters.crop", "polygon": polygon_wkt},

        # Pre hag filter
        pre_hag_filter,

        # Note, if you wanted to calculate your own ground classification points
        # do so here. We'll keep the ground classificaiton provided by VirtualTas.
        # e.g. { "type": "filters.csf", ... }
        # Calculate height above ground
        {"type": "filters.hag_nn"},
        # Label statistical outliers as noise (classification 7)
        {
            "type": "filters.outlier",
            "method": "statistical",
            "mean_k": 6,
            "multiplier": 10,
        },
      
        # Post hag filter for manual noise filtering
        post_hag_filter,

        # Load Z into Altitude and HeightAboveGround into Z
        {
            "type": "filters.ferry",
            "dimensions": "Z => Altitude, HeightAboveGround => Z"
        },
        
        # Classify any points below 0 as the ground as well
        {
            "type": "filters.assign",
            "value": "Z = 0 WHERE Z < 0",
        },

        # Save as a COPC file
        {
            "type": "writers.copc",
            "filename": output_path,
            "forward": "scale,offset",
            "extra_dims": "all",
        },

    ]

    pipeline_template_dict = filter(lambda x: x is not None, pipeline_template_dict)
    pipeline_template_dict = list(pipeline_template_dict)

    return json.dumps(pipeline_template_dict, indent=2)

### Site pipelines

We use some data from our previously created `sites.geojson` to create the pdal pipelines.

In [3]:
sites_gdf = gpd.read_file("../data/outputs/sites/sites.geojson")
sites_gdf = sites_gdf.set_index('id')

sites_gdf.head()

,site,geometry
id,,
AGG_O_01,AGG_O_01,"POLYGON ((463297.055 5259730.003, 463114.406 5..."
AGG_O_05,AGG_O_05,"POLYGON ((455424.932 5284132.819, 455198.808 5..."
AGG_O_07,AGG_O_07,"POLYGON ((464763.357 5299168.26, 464710.087 52..."
AGG_Y_02,AGG_Y_02,"POLYGON ((491861.764 5230973.15, 491841.78 523..."
AGG_Y_03,AGG_Y_03,"POLYGON ((490742.665 5208817.085, 490681.529 5..."


There are 3 variables in the pipeline above:
- input_path - Where to source the lidar file for that site
- output_path - Where to save the processed lidar file
- polygon_wkt - The polygon for that site in well known text (WKT) format
- manual_noise_filter_expression - An optional PDAL expression to filter noise points for cloud noise found in ULM_325 and ULM_147

In [4]:
def center_and_size_to_box(center, size):
    (cx, cy, cz) = center
    (sx, sy, sz) = size
    return (cx - 0.5 * sx, cx + 0.5 * sx,
            cy - 0.5 * sy, cy + 0.5 * sy,
            cz - 0.5 * sz, cz + 0.5 * sz)

def get_ulm_325_expr():
    # The values of the box were manually determined by inspecting the point cloud
    # in cloudcompare.
    box_center = (476110.031, 5230827.372, 87.567)
    box_size = (53.144, 49.606, 36.957)

    clip_box = center_and_size_to_box(box_center, box_size)
    (minx, maxx, miny, maxy, minz, maxz) = clip_box

    post_assign_expressions = [
        f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && HeightAboveGround >= {minz} && HeightAboveGround <= {maxz}",
        "Classification = 18 WHERE HeightAboveGround >= 100"
    ]

    return (None, post_assign_expressions)

def get_ulm_147_expr():
    # Pre clips use Z

    pre_box_center = (457928.866, 5285531.132, 417.412)
    pre_box_size = (89.205, 143.393, 32.591)
    pre_clip_box = center_and_size_to_box(pre_box_center, pre_box_size)
    (minx, maxx, miny, maxy, minz, maxz) = pre_clip_box

    pre_assign_expressions = [
       f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && Z >= {minz} && Z <= {maxz}",
    ]

    pre_box_center = (457940.068, 5285580.835, 544.315)
    pre_box_size = (69.96, 39.172, 38.754)
    pre_clip_box = center_and_size_to_box(pre_box_center, pre_box_size)
    (minx, maxx, miny, maxy, minz, maxz) = pre_clip_box

    pre_assign_expressions.append(
        f"Classification = 18 WHERE X >= {minx} && X <= {maxx} && Y >= {miny} && Y <= {maxy} && Z >= {minz} && Z <= {maxz}",
    )

    # Post clips can use HeightAboveGround

    boxA_center = (457919.822, 5285528.094, 121.494)
    boxA_size = (95.724, 93.489, 82.994)

    boxB_center = (457945.263, 5285589.062, 120.843)
    boxB_size = (58.202, 22.717, 29.792)

    clip_boxA = center_and_size_to_box(boxA_center, boxA_size)
    (minxa, maxxa, minya, maxya, minza, maxza) = clip_boxA

    clip_boxB = center_and_size_to_box(boxB_center, boxB_size)
    (minxb, maxxb, minyb, maxyb, minzb, maxzb) = clip_boxB

    post_assign_expressions = [
        f"Classification = 18 WHERE X >= {minxa} && X <= {maxxa} && Y >= {minya} && Y <= {maxya} && HeightAboveGround >= {minza} && HeightAboveGround <= {maxza}",
        f"Classification = 18 WHERE X >= {minxb} && X <= {maxxb} && Y >= {minyb} && Y <= {maxyb} && HeightAboveGround >= {minzb} && HeightAboveGround <= {maxzb}",
        "Classification = 18 WHERE HeightAboveGround >= 100"
    ]

    return (pre_assign_expressions, post_assign_expressions)


In [5]:
data_dir = Path("../data")
lidar_source_dir = data_dir / "source" / "cycle-2"  # cycle-2 has best coverage
lidar_output_dir = data_dir / "outputs" / "sites" / "lidar"
lidar_output_dir.mkdir(parents=True, exist_ok=True)


def create_pipeline_from_site(site_row):
    site_id = site_row.name

    input_path = str(lidar_source_dir / f"{site_id}.laz")
    output_path = str(lidar_output_dir / f"{site_id}.copc.laz")
    polygon_wkt = site_row.geometry.wkt
    manual_pre_norm_noise_expr = None
    manual_post_norm_noise_expr = None

    if site_id == "ULM_325":
        manual_pre_norm_noise_expr, manual_post_norm_noise_expr = get_ulm_325_expr()
    elif site_id == "ULM_147":
        manual_pre_norm_noise_expr, manual_post_norm_noise_expr = get_ulm_147_expr()

    return pd.Series(
        {
            "pipeline": create_site_pipeline(
                input_path=input_path,
                output_path=output_path,
                polygon_wkt=polygon_wkt,
                manual_pre_norm_noise_expr=manual_pre_norm_noise_expr,
                manual_post_norm_noise_expr=manual_post_norm_noise_expr,
            )
        }
    )


pipelines = sites_gdf.apply(create_pipeline_from_site, axis=1)

pipelines

,pipeline
id,
AGG_O_01,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_O_05,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_O_07,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_Y_02,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
AGG_Y_03,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
...,...
ULO_271,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
ULY_Y_231,"[\n {\n ""type"": ""readers.las"",\n ""filen..."
ULY_Y_232,"[\n {\n ""type"": ""readers.las"",\n ""filen..."


## Processing Pipelines

PDAL is built around processing these pipelines.

In [6]:
def process_pdal_pipeline(pipeline: str, return_data: bool = False):
    """
    Process a PDAL pipeline string.

    Args:
        pipeline (str): The PDAL pipeline JSON string.
        return_data (bool): If True, return the PDAL Pipeline object after execution. Defaults to False. Returning pipeline data
        will contain metadata and all the points processed by the pipeline. This can be a large object so defaults to False.
    """
    pipeline_obj = pdal.Pipeline(pipeline)
    count = pipeline_obj.execute()  # Execute the pipeline
    return (count, pipeline_obj if return_data else None)

Processing a single pipeline can take some time.

In [7]:
print(pipelines.loc['AGG_O_01'].pipeline)

[
  {
    "type": "readers.las",
    "filename": "../data/source/cycle-2/AGG_O_01.laz"
  },
  {
    "type": "filters.reprojection",
    "out_srs": "EPSG:7855+5711"
  },
  {
    "type": "filters.crop",
    "polygon": "POLYGON ((463297.05457586894 5259730.002777018, 463114.4059245781 5259760.099106976, 463066.3303332668 5259768.3808959965, 462963.0386275972 5259812.401819774, 462986.3186407427 5259876.65183454, 463209.48021875677 5259824.730678912, 463260.3091353938 5259811.636182331, 463260.76199426845 5259811.515737981, 463315.31154519285 5259796.550364946, 463297.05457586894 5259730.002777018))"
  },
  {
    "type": "filters.hag_nn"
  },
  {
    "type": "filters.outlier",
    "method": "statistical",
    "mean_k": 6,
    "multiplier": 10
  },
  {
    "type": "filters.ferry",
    "dimensions": "Z => Altitude, HeightAboveGround => Z"
  },
  {
    "type": "filters.assign",
    "value": "Z = 0 WHERE Z < 0"
  },
  {
    "type": "writers.copc",
    "filename": "../data/outputs/sites/lidar/A

In [8]:
%%time

p= pipelines.loc['ULM_147'].pipeline

(count, pl) = process_pdal_pipeline(p, return_data=True)
print(f"Processed {count} points.")

points = pl.arrays[0]
points_df = pd.DataFrame(pl.arrays[0])
points_df.head()

Processed 1482945 points.
CPU times: user 13.4 s, sys: 267 ms, total: 13.6 s
Wall time: 11.7 s


,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,Synthetic,...,UserData,PointSourceId,GpsTime,ScanChannel,Red,Green,Blue,Infrared,HeightAboveGround,Altitude
0,457926.829,5285501.429,6.273,32416,1,1,1,0,0,0,...,37,2,4.128239e+08,0,20817,20817,20046,33506,6.273,447.314
1,457926.989,5285501.368,34.723,29716,1,3,1,0,0,0,...,179,2,4.128239e+08,0,12079,14392,18247,18690,34.723,475.764
2,457927.269,5285501.316,37.566,29818,1,2,1,0,0,0,...,192,2,4.128239e+08,0,12336,14649,18504,16331,37.566,478.410
3,457927.083,5285501.348,37.968,30486,1,3,1,0,0,0,...,195,2,4.128239e+08,0,12336,14649,18504,17929,37.968,479.009
4,457927.339,5285501.290,49.112,30293,1,2,1,0,0,0,...,249,2,4.128239e+08,0,14649,16191,19275,20043,49.112,489.956


### Parallel processing with Dask

In [9]:
from dask.distributed import Client

client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60973,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:60984,Total threads: 2
Dashboard: http://127.0.0.1:60987/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:60976,


In [10]:
%%time

futures = client.map(process_pdal_pipeline, pipelines['pipeline'].to_list(), key=pipelines.index.to_list())
results = client.gather(futures)

CPU times: user 8.22 s, sys: 2.59 s, total: 10.8 s
Wall time: 4min 20s


In [11]:
total_points = 0
for r in results:
    total_points += r[0]

f"Total points: {total_points:,}"

'Total points: 88,866,965'

In [12]:
client.close()